The aim of this notebook is to create a vocabulary of chord types for chromas

We load data to see which chord types occur and define the vocab accordingly.

In [1]:
import collections
import matplotlib.pyplot as plt
import torch
from chord_progressions.type_templates import get_type_from_template, TYPE_TEMPLATES
from torch.utils.data import DataLoader
from tqdm import tqdm

from rhythmic_relationships.data import PairDataset

In [2]:
# Add silence
silence_template = '000000000000'
TYPE_TEMPLATES.update({'silence': silence_template})

# Rename out-of-vocab token and use silence to fill it
if '' in TYPE_TEMPLATES:
    del TYPE_TEMPLATES['']
TYPE_TEMPLATES['oov'] = silence_template

vocab_size = len(TYPE_TEMPLATES)
print(f'Vocab size: {vocab_size}')

inv_type_templates = {v: k for k, v in TYPE_TEMPLATES.items()}
stoi = {s:i for i,s in enumerate(inv_type_templates)}
itos = {i:s for s,i in stoi.items()}
itot = {ix:i for ix, i in enumerate(list(TYPE_TEMPLATES))}
ttoi = {v: k for k, v in itot.items()}

def pclist_to_i(pclist):
    """Gets a chord type index from a pitch class list
    e.g. pclist_to_i([1,0,0,0,0,0,0,0,0,0,0,0]) -> 1
    """
    chord_type = get_type_from_template(pclist)
    if not chord_type:
        chord_type = 'oov'
    template_str = TYPE_TEMPLATES[chord_type]
    return stoi[template_str]

Vocab size: 193


Ideally this notebook would be generalized to select a vocab depending on the dataset, but for now we use `lmdc_1000`.

In [3]:
# Load the dataset
dataset_config = {
    "dataset_name": "lmdc_1000_1bar_4res",
    "part_1": 'Guitar',
    "part_2": 'Piano',
    "repr_1": "chroma",
    "repr_2": "chroma",
}

data = PairDataset(**dataset_config)
loader = DataLoader(data, batch_size=1)

# Reduce the vocab size based on each token's occurence in the data
counts = collections.defaultdict(int)
for x, y in tqdm(loader):
    # Binarize the chromas
    x = (x > 1).to(torch.int32)[0]
    y = (y > 1).to(torch.int32)[0]

    for xrow, yrow in zip(x, y):
        ixx = pclist_to_i(xrow.tolist())
        ixy = pclist_to_i(yrow.tolist())
        counts[ixx] += 1
        counts[ixy] += 1

print(len(counts), {itot[k]: v for k,v in counts.items()})

100%|████████████████████████████████████████████████████████████████████████████████████████████| 22914/22914 [05:07<00:00, 74.41it/s]

83 {'silence': 372960, 'major chord': 70273, 'unison': 147009, 'perfect fourth': 29150, 'minor third': 10491, 'major third': 13813, 'minor chord': 29507, 'major-seventh chord': 6314, 'minor-seventh chord': 12701, 'whole-tone': 2440, 'incomplete minor-seventh chord': 3264, 'quartal tetramirror': 1866, 'augmented chord': 402, 'incomplete half-dim-seventh chord': 508, "french-sixth chord|messiaen's truncated 6": 78, 'dominant-seventh / german-sixth chord': 4295, 'dorian tetrachord|phrygian tetrachord': 83, 'major-ninth chord': 794, 'semitone': 633, 'diminished chord': 1673, 'tritone': 1025, 'lydian pentachord': 11, 'phrygian hexamirror': 9, 'balinese pelog pentatonic|korean': 3, 'major-second major tetrachord': 4188, 'phrygian trichord': 44, 'incomplete dominant-seventh chord 2': 2165, 'diminished-seventh chord': 232, 'quartal trichord': 7330, 'perfect-fourth major tetrachord': 859, 'minor-ninth chord': 717, 'major-second minor tetrachord': 344, 'minor trichord': 110, "natural / genuine /

In [4]:
sorted_type_counts = dict(sorted({itot[k]: v for k,v in counts.items()}.items(), key=lambda item: item[1]))

v_total = len(TYPE_TEMPLATES)
v_occur = len(sorted_type_counts)
pct_v_occur = round(v_occur * 100 / v_total, 1)
print(f'{v_total=}, {v_occur=} ({pct_v_occur}%)')

n_occur = sum(sorted_type_counts.values())
print(f'{n_occur=}')

v_total=193, v_occur=83 (43.0%)
n_occur=733248


In [5]:
# Define the percentage of occurences to be out-of-vocabulary
pct_oov = 0.02
n_oov = sum(counts.values()) * pct_oov

cur = 0
new_vocab_counts = {}
oov_counts = {}
for k,v in sorted_type_counts.items():
    cur += v
    if cur >= n_oov:
        new_vocab_counts[k] = v
    else:
        oov_counts[k] = v

v_in = len(new_vocab_counts)
v_out = len(oov_counts)

pct_v_in = round(v_in * 100 / v_occur, 1)
pct_v_out = round(v_out * 100 / v_occur, 1)
print(f'{v_in=} ({pct_v_in}%), {v_out=} ({pct_v_out}%)')

n_in = sum(new_vocab_counts.values())
n_out = sum(oov_counts.values())
pct_in = round(n_in * 100 / n_occur, 1)
pct_out = round(n_out * 100 / n_occur, 1)
print(f'{n_in=} ({pct_in}%), {n_out=} ({pct_out}%)')

v_in=17 (20.5%), v_out=66 (79.5%)
n_in=720051 (98.2%), n_out=13197 (1.8%)


In [6]:
# TODO: extend this remapping further
remap = {
 'minor-ninth chord': 'minor-seventh chord',
 'major-ninth chord': 'major-seventh chord',
 'diminished chord': 'tritone',
 'dominant-ninth,major-minor|prometheus pentamirror': 'tritone',
 'italian sixth|incomplete dominant-seventh chord 1': 'tritone',
 'dominanth-11th|natural / genuine / lydian hexachord': 'tritone',
 'pyramid': 'tritone',
}

remapped = collections.defaultdict(int)
for k, v in sorted_type_counts.items():
    if k in remap:
        k = remap[k]
    remapped[k] += v

cur = 0
remapped_new_vocab_counts = {}
remapped_oov_counts = {}
for k,v in remapped.items():
    cur += v
    if cur >= n_oov:
        remapped_new_vocab_counts[k] = v
    else:
        remapped_oov_counts[k] = v

v_in = len(remapped_new_vocab_counts)
v_out = len(remapped_oov_counts)

pct_v_in = round(v_in * 100 / v_occur, 1)
pct_v_out = round(v_out * 100 / v_occur, 1)
print(f'{v_in=} ({pct_v_in}%), {v_out=} ({pct_v_out}%)')

n_in = sum(remapped_new_vocab_counts.values())
n_out = sum(remapped_oov_counts.values())
pct_in = round(n_in * 100 / n_occur, 1)
pct_out = round(n_out * 100 / n_occur, 1)
print(f'{n_in=} ({pct_in}%), {n_out=} ({pct_out}%)')

v_in=18 (21.7%), v_out=59 (71.1%)
n_in=722421 (98.5%), n_out=10827 (1.5%)


In [7]:
# The condensed vocab list (for now)
list(remapped_new_vocab_counts)

['minor-seventh chord',
 'major-seventh chord',
 'perfect-fourth major tetrachord',
 'quartal tetramirror',
 'incomplete dominant-seventh chord 2',
 'half-diminished seventh chord',
 'whole-tone',
 'incomplete minor-seventh chord',
 'major-second major tetrachord',
 'dominant-seventh / german-sixth chord',
 'quartal trichord',
 'minor third',
 'major third',
 'perfect fourth',
 'minor chord',
 'major chord',
 'unison',
 'silence']